# Decoding of Next-Token Predictions

This note book is designed to reimplement Pathscope **section 1:Next-Token Prediction**

In [1]:
print("hello world")

hello world


In [2]:
import torch
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
print(torch.cuda.is_available())  # Should return True
print(torch.cuda.device_count())  # Should return the number of available GPUs
# print(torch.cuda.get_device_name(0))  # Should return the name of the GPU


True
1


In [4]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [111]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM,DataCollatorWithPadding
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.amp import autocast
import os
import math
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
# import dataloader


In [6]:
model_dict = ["Qwen/Qwen2-0.5B","Qwen/Qwen2.5-0.5B-Instruct","Qwen/Qwen2-1.5B"]

In [7]:
model_num = 0

tokenizer = AutoTokenizer.from_pretrained(model_dict[model_num])
model = AutoModelForCausalLM.from_pretrained(model_dict[model_num]).to(device)
model.eval()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [ ]:
# model(tokenizer('how are you doing today?'))

In [ ]:
print(model.__class__.__module__)
print(model.config.model_type)

transformers.models.qwen2.modeling_qwen2
qwen2


In [ ]:
type(model)

transformers.models.qwen2.modeling_qwen2.Qwen2ForCausalLM

In [ ]:
type(tokenizer)

transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast

In [ ]:
# from transformers import Qwen2ForCausalLM
# model2 = Qwen2ForCausalLM.from_pretrained('Qwen/Qwen2-0.5B')

In [ ]:
# print(model2)

In [8]:
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [9]:
# define a function to get hidden states of each layer of the model

def get_hidden_state(model, input_ids):
    with torch.no_grad():
        outputs = model(input_ids.to(device), output_hidden_states=True)
    return outputs.hidden_states[1:]  #Size([batch_size, num_layers, sequence_length, hidden_size])




In [10]:
# load the Pile data "https://huggingface.co/datasets/EleutherAI/pile" from hugging face, unfortunately it is not available
# use the following uncoprighted pile dataset instead
# https://huggingface.co/datasets/monology/pile-uncopyrighted

dataPile= load_dataset("monology/pile-uncopyrighted",split='train',streaming=True)


README.md:   0%|          | 0.00/776 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [11]:
dataPile

IterableDataset({
    features: Unknown,
    num_shards: 30
})

In [12]:
# shuffle the dataset before spliting them
# reference resource "https://huggingface.co/docs/datasets/v3.3.2/stream"

dataPile = dataPile.shuffle(seed=50)
training_data = dataPile.take(10000)
# train : val = 5 : 1
validation_data = dataPile.skip(10000).take(2000)

In [13]:
list(validation_data)[:4]

[{'text': 'The present invention relates generally to the art of sputtering silicon-containing target materials in a reactive atmosphere, and more particularly to reactive sputtering of silicon targets further comprising a transition metal.\nGerman Patent Application DE 3,906,453 Al to Szczybowski et al. discloses sputtering silicides, such as nickel silicide (NiSi.sub.2), in an oxidizing atmosphere to deposit dielectric oxide films.\nThe present invention involves silicon-nickel alloys ranging from 3 to 18 weight percent nickel. Targets of silicon-nickel alloys are sputtered in an atmosphere comprising nitrogen, oxygen, inert gases and mixtures thereof to produce silicon-nickel containing coatings including oxides, nitrides and oxynitrides. The silicon-nickel compositions of the present invention comprise sufficient nickel to provide target stability and a desirable sputtering rate while keeping the absorption of the resulting films relatively low.',
  'meta': {'pile_set_name': 'USPTO

In [14]:
text = "1. Field of the Invention\nThe present invention relates to toothbrushes and, in particular, to a toothbrush having a hollow handle defining a paste-holding cavity wherein toothpaste is forcibly dispensed therefrom"

In [15]:
tokenizer([text,'hee asdf po afsdf '], return_tensors="pt",max_length=21, truncation=True, padding="max_length")

{'input_ids': tensor([[    16,     13,   8601,    315,    279,    758,   7459,    198,    785,
           3042,  27130,  35616,    311,  25507,  36061,    288,    323,     11,
            304,   3953,     11],
        [ 49053,    438,   2940,   3193,  66713,   2940,    220, 151643, 151643,
         151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643,
         151643, 151643, 151643]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])}

In [16]:

import random

def preprocess(example):

    text = example['text']
    inputs = tokenizer(text, return_tensors='pt', max_length=50, truncation=True) # dict_keys(['input_ids', 'attention_mask'])
    inputs_ids = inputs['input_ids'][0] # becasue the batch size is 1

    if len(inputs_ids) > 5:
        trim_len = random.randint(5,  len(inputs_ids) -1)   # to introduce randomness in the training data
        inputs_ids = inputs_ids[:trim_len]

    return {'input_ids': inputs_ids, 'attention_mask': inputs['attention_mask'][0][:len(inputs_ids)]}

In [17]:
training_data = training_data.map(preprocess,remove_columns=['text','meta'])
validation_data = validation_data.map(preprocess,remove_columns=['text','meta'])

In [18]:
next(iter(training_data))

{'input_ids': tensor([   35,   321,  1345,   496,   220,    16,    24,    24,    23,   271,
         38704, 10099,   279,  2480,  5700]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])}

In [19]:
# design a collate function

In [20]:
# we can change the attentino mask to focus on the token of interest

In [21]:

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [22]:
# PyTorch
torch.cuda.memory_summary()

# Or more detailed
torch.cuda.memory_allocated()
torch.cuda.memory_reserved()


2097152000

## 1. Logit lens

In [26]:
# each_layer_logit_lens = {l : [] for l in range(len(24))}
from collections import defaultdict
from tqdm import tqdm

each_layer_logit_lens_Precision = defaultdict(list) # key = int:layer , value = list[]
each_layer_logit_lens_Surprisal = defaultdict(list)

def logit_len_for_test(hidden_states,refernce_logits):
    pass



def logit_len(hidden_states,mask,refernce_logits):
    """

    hidden_states: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model

    mask: Size([batch_size, sequence_length])

    refernce_logits: Size([batch_size, sequence_length, vocab_size]): this will be converted to Size([batch_size, vocab_size])

    """
    W_U = model.lm_head.weight

    # using attention mask to get the last token of the sequence

    num_layer  = len(hidden_states)
    batch_size, sequence_length, hidden_size = hidden_states[0].shape

    last_token_pos = torch.sum(mask,dim=1) # Size([batch_size])

    reference_logits = refernce_logits[torch.arange(len(mask)),last_token_pos -1 ,:] # Size([batch_size, vocab_size])
    reference_logits_max_token = torch.argmax(reference_logits,dim = -1)

    def suprisla(reference_logist, logist):
        predicted_tokens = torch.argmax(logits, dim=-1)  # Shape: (batch_size,)
        reference_probs = torch.nn.functional.softmax(reference_logits, dim=-1)
        predicted_token_probs = reference_probs[torch.arange(batch_size), predicted_tokens]
        surprisal = -torch.log(predicted_token_probs + 1e-9)

        return torch.sum(surprisal)



    for idx, each_layer in enumerate(hidden_states):
        layer_hidden = each_layer[torch.arange(batch_size),last_token_pos-1] # Size([batch_size, hidden_size])
        # layer_hidden = layer_hidden[torch.arange(batch_size),:] # Size([batch_size, hidden_size])
        logits = torch.einsum('bd,vd->bv', layer_hidden, W_U)  # Size([batch_size, vocab_size])
        logits_max_token = torch.argmax(logits,dim = -1)

        identical_samples = torch.sum(logits_max_token == reference_logits_max_token)
        each_layer_logit_lens_Precision[idx].append(identical_samples)
        each_layer_logit_lens_Surprisal[idx].append(suprisla(reference_logits,logits))
        # print(identical_samples)


    return

batch_size = 32

i = 0
for batch in tqdm(DataLoader(validation_data, batch_size=batch_size, collate_fn=data_collator),total = mach.ceil(2000/batch_size),desc = f"logit_lens_exp"):
    # print(batch)k
    batch.to(device)
    with torch.no_grad():
        model_outputs = model(**batch,output_hidden_states=True)
        hidden_states = model_outputs.hidden_states[1:]  # exclude the embedding layer
        reference_logits = model_outputs.logits
        mask = batch['attention_mask']

    # reference_logits = reference_logits[torch.arange(len(mask)),torch.sum(mask,dim=1)] # Size([batch_size, vocab_size])
    Compare_logit = logit_len(hidden_states,mask,reference_logits)
    # i += 1
    # print('one batch done')

    # if i >= 2:
    #     break

logit_lens_exp: 63it [00:58,  1.07it/s]


In [25]:
device

device(type='cuda')

In [ ]:
# Clear GPU cache
torch.cuda.empty_cache()

In [30]:
each_layer_logit_lens_Precision

defaultdict(list,
            {0: [tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(1, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(1, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(1, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(1, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(1, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(0, device='cuda:0'),
              tensor(2, device='cuda:0'),
             

In [31]:
print(# PyTorch
torch.cuda.memory_summary())

# Or more detailed
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   2371 MiB |   4054 MiB | 474818 MiB | 472447 MiB |
|       from large pool |   2332 MiB |   3957 MiB | 461808 MiB | 459475 MiB |
|       from small pool |     38 MiB |     98 MiB |  13010 MiB |  12971 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   2371 MiB |   4054 MiB | 474818 MiB | 472447 MiB |
|       from large pool |   2332 MiB |   3957 MiB | 461808 MiB |

In [32]:
! nvidia-smi

Mon Mar 24 21:23:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   65C    P0             30W /   70W |    5382MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

#### Metric
1. Precision@1
2. Suprisal

In [33]:
def precision(a):
    """
    a : dict[layer] =  list[]
    """

    for k, v in a.items():
        a[k] = torch.stack(v,dim = 0).sum() / 2000

    return a


def surprisal(a):

    for k,v in a.items():
        a[k] = torch.stack(v,dim = 0).sum()/2000

    return a

logit_len_p = precision(each_layer_logit_lens_Precision)
logit_len_s = surprisal(each_layer_logit_lens_Surprisal)

In [34]:
logit_len_p

defaultdict(list,
            {0: tensor(0.0090, device='cuda:0'),
             1: tensor(0.0020, device='cuda:0'),
             2: tensor(0.0020, device='cuda:0'),
             3: tensor(0.0030, device='cuda:0'),
             4: tensor(0.0045, device='cuda:0'),
             5: tensor(0.0035, device='cuda:0'),
             6: tensor(0.0045, device='cuda:0'),
             7: tensor(0.0035, device='cuda:0'),
             8: tensor(0.0060, device='cuda:0'),
             9: tensor(0.0050, device='cuda:0'),
             10: tensor(0.0045, device='cuda:0'),
             11: tensor(0.0090, device='cuda:0'),
             12: tensor(0.0135, device='cuda:0'),
             13: tensor(0.0170, device='cuda:0'),
             14: tensor(0.0240, device='cuda:0'),
             15: tensor(0.0220, device='cuda:0'),
             16: tensor(0.0680, device='cuda:0'),
             17: tensor(0.0725, device='cuda:0'),
             18: tensor(0.0725, device='cuda:0'),
             19: tensor(0.1015, device='cu

In [35]:
print(model_outputs['logits'].shape)
print(model_outputs['hidden_states'][4].shape)



torch.Size([16, 43, 151936])
torch.Size([16, 43, 896])


In [36]:
len(model_outputs['hidden_states'])

25

## 2. Tuned Lens

In [38]:
from sklearn.linear_model import LinearRegression

import numpy as np


config = model.config
num_layers = config.num_hidden_layers

X_train = {l : [] for l in range(num_layers)}
Y_train = []  # finaly layer logits



batch_size = 32

def get_last_token_hidden_states(hidden_states,mask) -> None:
    """
    hiddenstates: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model
    """

    last_token_pos = torch.sum(mask,dim=1) # Size([batch_size])

    for i, layer in enumerate(hidden_states):
        X_train[i].append(layer[torch.arange(len(mask)),last_token_pos -1 ,:].cpu().numpy())   # Size([batch_size, hidden_size])

    Y_train.append(hidden_states[-1][torch.arange(len(mask)),last_token_pos -1 ,:].cpu().numpy())   # Size([batch_size, hidden_size]))

# to get the hidden states of each sequence
for batch in tqdm(DataLoader(training_data, batch_size=batch_size, collate_fn=data_collator),total = math.ceil(10000 / batch_size) ,desc = "affine map training data"):
    batch.to(device)
    with torch.no_grad():
        model_outputs = model(**batch,output_hidden_states=True)
    # batch.to('cpu')
    hidden_states = model_outputs.hidden_states[1:] # exclude the embedding layer
    mask = batch['attention_mask'].cpu()
    reference_logits = model_outputs.logits

    get_last_token_hidden_states(hidden_states,mask)

#     del batch, model_outputs
#     torch.cuda.empty_cache()  # Clear GPU cache



affine map training data: 100%|██████████| 313/313 [03:30<00:00,  1.48it/s]


In [42]:
X_train[0][-1].shape

(16, 896)

In [41]:
len(Y_train)

313

In [43]:
# for each layer,train a affing transformation to predict the final layer logits

Y_train = np.concatenate(Y_train,axis=0) # Size([num_samples, hidden_size])

affine_map_house = []

for i in range(num_layers):
    affine_map = LinearRegression(fit_intercept=True)
    X_train[i] = np.concatenate(X_train[i],axis=0) # Size([num_samples, hidden_size])
    affine_map.fit(X_train[i],Y_train)
    affine_map_house.append(affine_map)
    # break



In [48]:

# do the same thing as logit_len but with the affine transformation bewteen the hidden states of each layer and the final layer logits
each_layer_tuned_lens_Precision = defaultdict(list)
each_layer_tuned_lens_Surprisal = defaultdict(list)

def tuned_len(hidden_states,mask,refernce_logits,model = model):
    """

    hidden_states: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model

    mask: Size([batch_size, sequence_length])

    refernce_logits: Size([batch_size, sequence_length, vocab_size]): this will be converted to Size([batch_size, vocab_size])

    """
    W_U = model.lm_head.weight

    # using attention mask to get the last token of the sequence

    num_layer  = len(hidden_states)
    batch_size, sequence_length, hidden_size = hidden_states[0].shape

    last_token_pos = torch.sum(mask,dim=1) # Size([batch_size])

    reference_logits = refernce_logits[torch.arange(len(mask)),last_token_pos -1 ,:] # Size([batch_size, vocab_size])
    reference_logits_max_token = torch.argmax(reference_logits,dim = -1)

    def suprisla(reference_logist, logist):
        predicted_tokens = torch.argmax(logits, dim=-1)  # Shape: (batch_size,)
        reference_probs = torch.nn.functional.softmax(reference_logits, dim=-1)
        predicted_token_probs = reference_probs[torch.arange(batch_size), predicted_tokens]
        surprisal = -torch.log(predicted_token_probs + 1e-9)

        return torch.sum(surprisal)


    for i , each_layer in enumerate(hidden_states):
        layer_hidden = each_layer[torch.arange(batch_size),last_token_pos-1] # Size([batch_size, hidden_size]

        #***************************************************************
        # the only changes compared to logit_len
        affine_map = affine_map_house[i]   #这里竟然没有报错，当我只有第一个layer的affine map的时候
        layer_hidden_mapped = affine_map.predict(layer_hidden.cpu().numpy())
        layer_hidden_mapped = torch.tensor(layer_hidden_mapped).to(device)



        #***************************************************************
        logits = torch.einsum('bd,vd->bv', layer_hidden_mapped, W_U)  # Size([batch_size, vocab_size])
        logits_max_token = torch.argmax(logits,dim = -1)

        identical_samples = torch.sum(logits_max_token == reference_logits_max_token)
        each_layer_tuned_lens_Precision[i].append(identical_samples)
        each_layer_tuned_lens_Surprisal[i].append(suprisla(reference_logits,logits))
        # print(identical_samples)
        # print(torch.sum(logits_max_token == reference_logits_max_token))


batch_size = 32

i = 0
for batch in tqdm(DataLoader(validation_data, batch_size=batch_size, collate_fn=data_collator),total = math.ceil(2000/batch_size),desc = "tuned_len_exp"):
    # print(batch)k
    batch.to(device)
    with torch.no_grad():
        model_outputs = model(**batch,output_hidden_states=True)
        hidden_states = model_outputs.hidden_states[1:]  # exclude the embedding layer
        reference_logits = model_outputs.logits
        mask = batch['attention_mask']

    # reference_logits = reference_logits[torch.arange(len(mask)),torch.sum(mask,dim=1)] # Size([batch_size, vocab_size])
    Compare_logit = tuned_len(hidden_states,mask,reference_logits)
    i += 1
    # print('one batch done')

    # if i >= 5:
    #     break

tuned_len_exp: 100%|██████████| 63/63 [00:57<00:00,  1.09it/s]


In [49]:
tuned_len_p = precision(each_layer_tuned_lens_Precision)
tuned_len_s = surprisal(each_layer_tuned_lens_Surprisal)

In [97]:
tuned_len_p

defaultdict(list,
            {0: tensor(0.1730, device='cuda:0'),
             1: tensor(0.1715, device='cuda:0'),
             2: tensor(0.1780, device='cuda:0'),
             3: tensor(0.2030, device='cuda:0'),
             4: tensor(0.2170, device='cuda:0'),
             5: tensor(0.2120, device='cuda:0'),
             6: tensor(0.2085, device='cuda:0'),
             7: tensor(0.2215, device='cuda:0'),
             8: tensor(0.2215, device='cuda:0'),
             9: tensor(0.2320, device='cuda:0'),
             10: tensor(0.2350, device='cuda:0'),
             11: tensor(0.2510, device='cuda:0'),
             12: tensor(0.2640, device='cuda:0'),
             13: tensor(0.2790, device='cuda:0'),
             14: tensor(0.2930, device='cuda:0'),
             15: tensor(0.3110, device='cuda:0'),
             16: tensor(0.4000, device='cuda:0'),
             17: tensor(0.4430, device='cuda:0'),
             18: tensor(0.4770, device='cuda:0'),
             19: tensor(0.5265, device='cu

In [50]:
tuned_len_s

defaultdict(list,
            {0: tensor(5.4510, device='cuda:0'),
             1: tensor(5.2155, device='cuda:0'),
             2: tensor(5.1047, device='cuda:0'),
             3: tensor(4.8496, device='cuda:0'),
             4: tensor(4.6575, device='cuda:0'),
             5: tensor(4.6832, device='cuda:0'),
             6: tensor(4.6624, device='cuda:0'),
             7: tensor(4.5858, device='cuda:0'),
             8: tensor(4.5835, device='cuda:0'),
             9: tensor(4.5126, device='cuda:0'),
             10: tensor(4.4588, device='cuda:0'),
             11: tensor(4.3821, device='cuda:0'),
             12: tensor(4.2611, device='cuda:0'),
             13: tensor(4.1287, device='cuda:0'),
             14: tensor(4.0046, device='cuda:0'),
             15: tensor(3.8523, device='cuda:0'),
             16: tensor(3.2360, device='cuda:0'),
             17: tensor(2.9114, device='cuda:0'),
             18: tensor(2.6892, device='cuda:0'),
             19: tensor(2.4624, device='cu

In [ ]:
X_train[0].shape

In [ ]:
Y_train.shape

we can conclude that tuned lens really outputperform vallina logit lean

## 3.Token Identitiy Patchscope

set the hidden state at the specic layer and position to the last

### A. Construct **Few-Shot Token Identity Prompts**

In [51]:
# sample a random set of k tokens for all the models, where k was also randomly sampled from the interval [1, . . . , 10].

# to construct 5 realization of token iDs series of variable length

def craft_realization() -> torch.Tensor:
    """
    return: Size([num_realization, num_tokens])
    """

    identity_prompts = []

    num_realization = 5

    for j in range(num_realization):
        num_tokens = random.sample(range(tokenizer.vocab_size),random.randint(1,10))
        prompt = ';'.join([ f"{tokenizer.decode([i])} -> {tokenizer.decode([i])}"  for i in num_tokens]) + '; ?'
        identity_prompts.append(prompt)


    return identity_prompts

identity_token_prompts = craft_realization()




In [52]:
print(identity_token_prompts)

['漯 -> 漯; next ->  next;Gam -> Gam;_draft -> _draft;创造力 -> 创造力;(GPIO -> (GPIO;眼看 -> 眼看;悩 -> 悩;ipient -> ipient; Specifically ->  Specifically; ?', 'egra -> egra; abstraction ->  abstraction;.sav -> .sav;بريط -> بريط;createQuery -> createQuery;-moving -> -moving; contenders ->  contenders; đừng ->  đừng;uncia -> uncia; ?', 'Them -> Them;锅炉 -> 锅炉; prompts ->  prompts;\tUPROPERTY -> \tUPROPERTY; imprimir ->  imprimir; מאשר ->  מאשר;ämpfe -> ämpfe; flexible ->  flexible; ?', '镃 -> 镃; יכול ->  יכול;早早 -> 早早;某某 -> 某某;.viewport -> .viewport; ?', '的最佳 -> 的最佳;чи -> чи; dood ->  dood;Fetching -> Fetching; migliori ->  migliori;\tSpring -> \tSpring;เนอร์ -> เนอร์;慷 -> 慷; ?']


In [62]:
# tokenizer.tokenize(identity_token_prompts[0])

In [53]:
model.model.layers

ModuleList(
  (0-23): 24 x Qwen2DecoderLayer(
    (self_attn): Qwen2Attention(
      (q_proj): Linear(in_features=896, out_features=896, bias=True)
      (k_proj): Linear(in_features=896, out_features=128, bias=True)
      (v_proj): Linear(in_features=896, out_features=128, bias=True)
      (o_proj): Linear(in_features=896, out_features=896, bias=False)
    )
    (mlp): Qwen2MLP(
      (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
      (up_proj): Linear(in_features=896, out_features=4864, bias=False)
      (down_proj): Linear(in_features=4864, out_features=896, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
    (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
  )
)

In [96]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2RotaryEmbe

In [106]:
debug_idx = 23
def patch_hidden_state(source_hidden_states : tuple,batch_length:torch.Tensor, model=model,target_prompt = None) -> None:
    """
    source_hidden_states: Tuple[Tensor:Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model
    batch_length: Size([batch_size])

    target_prompt: str: the prompt the patched hidden states should be conditioned on

    """
    patched_results = {l : [] for l in range(len(source_hidden_states))}


    for layer_index, layer in enumerate(source_hidden_states):

        layer = layer[torch.arange(len(batch_length)),batch_length -1 ,:] # Size([batch_size, hidden_size])


        for i in range(layer.shape[0]):

            hs_to_be_patched = layer[i]

            # set a farward hook function
            def patch_hidden_hook(module, input, output):
                """
                input: Tuple[Tensor:Size([batch_size, sequence_length, hidden_size])]
                output: Tuple[Size([batch_size, sequence_length, hidden_size])]: the hidden states of each layer of the model

                """
                # print(f' ')
                output[0][0,-1] = hs_to_be_patched

                if layer_index == debug_idx:  # for debugging reason
                    print(output[0][0,-1][:6])


            # set a hook to the target layer
            hook_handle = model.model.layers[layer_index].register_forward_hook(patch_hidden_hook)

            with torch.no_grad():
                output_Target = model(**tokenizer(target_prompt,return_tensors='pt').to(device),output_hidden_states=True)
                target_logits = output_Target.logits[0][-1]
                patched_results[layer_index].append(target_logits)

            hook_handle.remove()



            if layer_index == debug_idx:  # for debugging reason

                print(output_Target.hidden_states[debug_idx + 1 ][0][-1][:6])


    for k,v in patched_results.items():
        patched_results[k] = torch.stack(v)

    return patched_results



In [107]:
tokenizer(identity_token_prompts[0],return_tensors='pt',padding=True)['input_ids'].shape


torch.Size([1, 57])

In [108]:
def cal_precision_1(source, target,sources_length):
    """
    source: Size([batch_size, sequence_len, vocab_size])
    target: Dict(K: num_layer, V: Size([batch_size, vocab_size]))

    """

    source = source[torch.arange(len(sources_length)),sources_length -1 ,:] # Size([batch_size, vocab_size])

    for i in range(len(target)):
        print(torch.sum(torch.argmax(source,dim=-1) == torch.argmax(target[i],dim=-1),dim = 0))

In [113]:
for prompt in identity_token_prompts:

    # get the hidden satate of the prompt at the last positon


    batch_size = 4
    for batch in DataLoader(validation_data, batch_size=batch_size ,collate_fn=data_collator): #,total = (2000/ batch_size), desc = "patchscope_exp"):
        # print(batch)
        batch.to(device)

        # run the first forward pass on the source prompt
        with torch.no_grad():
            # with autocast('cuda'):
            # print("hello 123")
            model_outputs = model(**batch,output_hidden_states=True)
            # print("heelo ")
            hidden_states = model_outputs.hidden_states[1:]
            reference_logits = model_outputs.logits # Size([batch_size, sequence_length, vocab_size])
            batch_length = torch.sum(batch['attention_mask'],dim=1)

        target_logtis = patch_hidden_state(hidden_states,batch_length,target_prompt=prompt)

        # print(hidden_states[0][:,batch_length,:][:6])
        cal_precision_1(reference_logits,target_logtis,batch_length)
        # batch.to('cpu')
        # del batch, model_outputs
        # torch.cuda.empty_cache()

        break
    break

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [95]:
# to check at the last layer if the logits are the same as the target logits